<a href="https://colab.research.google.com/github/DhrubaAdhikary/GEN_AI_DEMO/blob/master/Conversational_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import google.colab.output
from IPython.display import HTML, display

# 1. Define the Python function that the UI will call
def handle_chat(user_input):
    # This is where your LangGraph app logic goes
    try:
        # Simulate state for testing - replace with: state = app.invoke(...)
        mock_response = f"Simulated Response for: {user_input}"
        mock_intent = "check_order"

        # This returns HTML to be appended to the chat log
        return f"""
        <div style="margin-bottom: 10px;">
          <b>User:</b> {user_input}<br>
          <span style="color: teal;"><b>NLU:</b> {mock_intent}</span><br>
          <b>Agent:</b> {mock_response}
          <hr>
        </div>
        """
    except Exception as e:
        return f"<div style='color:red'>Error: {str(e)}</div>"

# Register the function so JavaScript can see it
google.colab.output.register_callback('notebook.handle_chat', handle_chat)

# 2. Build the HTML/JS Interface
chat_ui = """
<div id="chat-container" style="font-family: sans-serif; border: 1px solid #ccc; padding: 15px; border-radius: 8px; background: #f9f9f9;">
    <div id="log" style="height: 200px; overflow-y: auto; background: white; border: 1px solid #ddd; padding: 10px; margin-bottom: 10px;">
        <i>Chat history will appear here...</i>
    </div>
    <input type="text" id="user-input" style="width: 70%; padding: 5px;" placeholder="Ask about your order...">
    <button onclick="sendToPython()" style="padding: 5px 15px; cursor: pointer; background: #28a745; color: white; border: none; border-radius: 4px;">Send</button>
</div>

<script>
async function sendToPython() {
    const input = document.getElementById('user-input');
    const log = document.getElementById('log');
    const text = input.value;
    if (!text) return;

    input.value = ''; // Clear input

    // Call the Python function we registered
    const result = await google.colab.kernel.invokeFunction('notebook.handle_chat', [text], {});

    // Append the result to our chat log
    log.innerHTML += result.data['text/plain'];
    log.scrollTop = log.scrollHeight; // Auto-scroll
}
</script>
"""

display(HTML(chat_ui))

In [ ]:
!pip install -qU langgraph langchain_openai ipywidgets nltk
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Order-Status-Orchestrator"

In [ ]:
# import operator
# from typing import Annotated, List, TypedDict, Union
# from langchain_openai import ChatOpenAI
# from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
# from langgraph.graph import StateGraph, START, END

# # 1. State Definition
# class AgentState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     order_id: Union[str, None]
#     sentiment: str
#     intent: str

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# # 2. NLU Node: Extracts Intent, Sentiment, and Entities (Order ID)
# def nlu_node(state: AgentState):
#     last_message = state["messages"][-1].content
#     # Few-Shot instruction for NLU
#     prompt = f"""Analyze the user input: "{last_message}"
#     Extract:
#     - Intent (e.g., check_order, greeting)
#     - Sentiment (Positive, Negative, Neutral)
#     - Order ID (if mentioned, otherwise None)
#     Return as: Intent | Sentiment | OrderID"""

#     res = llm.invoke(prompt).content.split(" | ")
#     return {
#         "intent": res[0],
#         "sentiment": res[1],
#         "order_id": res[2] if "None" not in res[2] else None
#     }

# # 3. Dialogue Manager logic (The Router)
# def dialogue_manager_router(state: AgentState):
#     if state["order_id"] is None:
#         return "ask_for_id"
#     return "retrieve_order"

# # 4. Clarification Node (Missing Entity)
# def ask_for_id_node(state: AgentState):
#     return {"messages": [AIMessage(content="I can help with that! What is your order number?")]}

# # 5. RAG Retriever Node (Context & Prompt)
# def retrieve_order_node(state: AgentState):
#     # Simulated Database Lookup based on 'Order Database' in image
#     order_info = f"Order {state['order_id']} has shipped and will arrive tomorrow."
#     return {"messages": [AIMessage(content=order_info)]}

# # --- Build the Graph ---
# workflow = StateGraph(AgentState)
# workflow.add_node("nlu", nlu_node)
# workflow.add_node("ask_for_id", ask_for_id_node)
# workflow.add_node("retrieve_order", retrieve_order_node)

# workflow.add_edge(START, "nlu")
# workflow.add_conditional_edges("nlu", dialogue_manager_router)
# workflow.add_edge("ask_for_id", END)
# workflow.add_edge("retrieve_order", END)

# app = workflow.compile()



In [ ]:
import os
import operator
import google.colab.output
from typing import Annotated, List, TypedDict, Union
from IPython.display import HTML, display

# 1. DISABLE LANGSMITH TRACING (Removes the 401 Warnings)
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')

# 2. Define the Python function that the UI will call

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END

# --- Graph Logic with Robust Parsing ---
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    order_id: Union[str, None]
    sentiment: str
    intent: str

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def nlu_node(state: AgentState):
    last_message = state["messages"][-1].content
    prompt = f"""Analyze: "{last_message}"
    Return ONLY: Intent | Sentiment | OrderID (use None if missing)"""

    try:
        res_raw = llm.invoke(prompt).content
        # Split and clean whitespace
        parts = [p.strip() for p in res_raw.split("|")]

        # Defensive check: ensure list has at least 3 items
        intent = parts[0] if len(parts) > 0 else "unknown"
        sentiment = parts[1] if len(parts) > 1 else "Neutral"
        oid = parts[2] if len(parts) > 2 else "None"

        return {
            "intent": intent,
            "sentiment": sentiment,
            "order_id": oid if "None" not in oid else None
        }
    except Exception:
        return {"intent": "error", "sentiment": "Neutral", "order_id": None}

# ... (rest of your nodes: ask_for_id_node, retrieve_order_node, and dialogue_manager_router remain the same) ...

def dialogue_manager_router(state: AgentState):
    return "ask_for_id" if state["order_id"] is None else "retrieve_order"

def ask_for_id_node(state: AgentState):
    return {"messages": [AIMessage(content="I can help with that! What is your order number?")]}

def retrieve_order_node(state: AgentState):
    order_info = f"Order {state['order_id']} has shipped and will arrive tomorrow."
    return {"messages": [AIMessage(content=order_info)]}

workflow = StateGraph(AgentState)
workflow.add_node("nlu", nlu_node)
workflow.add_node("ask_for_id", ask_for_id_node)
workflow.add_node("retrieve_order", retrieve_order_node)
workflow.add_edge(START, "nlu")
workflow.add_conditional_edges("nlu", dialogue_manager_router)
workflow.add_edge("ask_for_id", END)
workflow.add_edge("retrieve_order", END)
app = workflow.compile()

# --- The UI Bridge ---
def handle_chat(user_input):
    try:
        state = app.invoke({"messages": [HumanMessage(content=user_input)], "order_id": None})
        response = state["messages"][-1].content
        return f"<div><b>User:</b> {user_input}<br><small>NLU: {state['intent']}</small><br><b>Agent:</b> {response}<hr></div>"
    except Exception as e:
        return f"<div style='color:red'>Error: {str(e)}</div>"

google.colab.output.register_callback('notebook.handle_chat', handle_chat)
display(HTML(chat_html)) # Uses the same chat_html from our previous step